In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

#### Criar sessão PySpark

In [4]:
spark = (
  SparkSession.builder
  .master('local')
  .appName('PySpark_01')
  .getOrCreate()
)

#### Criar DF / ler arquivo

In [5]:
df = spark.read.csv('./netflix_titles.csv', header=True, inferSchema=True)

#### Exibir DF

In [6]:
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         NULL|Septem

#### Verificar tipos de colunas

In [7]:
df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



#### Verificando dados nulos

In [8]:
for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

show_id 0
type 1
title 2
director 2636
cast 826
country 832
date_added 13
release_year 2
rating 6
duration 5
listed_in 3
description 3


#### Renomeando Colunas

In [9]:
portDf = df.withColumnRenamed('show_id', 'id')\
.withColumnRenamed('type', 'tipo')\
.withColumnRenamed('title', 'titulo')\
.withColumnRenamed('director', 'diretor')\
.withColumnRenamed('cast' ,'atores_principais')\
.withColumnRenamed('country', 'pais')\
.withColumnRenamed('date_added', 'data_dicionado')\
.withColumnRenamed('release_year', 'ano_lancamento')\
.withColumnRenamed('rating', 'classificacao')\
.withColumnRenamed('duration', 'duracao')\
.withColumnRenamed('listed_in', 'categoria')\
.withColumnRenamed('description', 'descricao').show(5)

+---+-------+--------------------+---------------+--------------------+-------------+------------------+--------------+-------------+---------+--------------------+--------------------+
| id|   tipo|              titulo|        diretor|   atores_principais|         pais|    data_dicionado|ano_lancamento|classificacao|  duracao|           categoria|           descricao|
+---+-------+--------------------+---------------+--------------------+-------------+------------------+--------------+-------------+---------+--------------------+--------------------+
| s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|          2020|        PG-13|   90 min|       Documentaries|As her father nea...|
| s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|          2021|        TV-MA|2 Seasons|International TV ...|After crossing pa...|
| s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...

#### Selecionar Colunas

In [10]:
df.select('title', 'description').show(5)

+--------------------+--------------------+
|               title|         description|
+--------------------+--------------------+
|Dick Johnson Is Dead|As her father nea...|
|       Blood & Water|After crossing pa...|
|           Ganglands|To protect his fa...|
|Jailbirds New Orl...|Feuds, flirtation...|
|        Kota Factory|In a city of coac...|
+--------------------+--------------------+
only showing top 5 rows



In [11]:
df.select(col('title'), col('description')).show(5)

+--------------------+--------------------+
|               title|         description|
+--------------------+--------------------+
|Dick Johnson Is Dead|As her father nea...|
|       Blood & Water|After crossing pa...|
|           Ganglands|To protect his fa...|
|Jailbirds New Orl...|Feuds, flirtation...|
|        Kota Factory|In a city of coac...|
+--------------------+--------------------+
only showing top 5 rows



In [12]:
df.select(df['title']).show(5)

+--------------------+
|               title|
+--------------------+
|Dick Johnson Is Dead|
|       Blood & Water|
|           Ganglands|
|Jailbirds New Orl...|
|        Kota Factory|
+--------------------+
only showing top 5 rows



#### Selecionar Colunas com ALIAS
**Pelo primeiro método de select, o alias não funciona por se tratar de uma string**

In [13]:
df.select(col('title').alias('titulo')).show(5)

+--------------------+
|              titulo|
+--------------------+
|Dick Johnson Is Dead|
|       Blood & Water|
|           Ganglands|
|Jailbirds New Orl...|
|        Kota Factory|
+--------------------+
only showing top 5 rows



In [14]:
df.select('title director cast'.split()).show(5)

+--------------------+---------------+--------------------+
|               title|       director|                cast|
+--------------------+---------------+--------------------+
|Dick Johnson Is Dead|Kirsten Johnson|                NULL|
|       Blood & Water|           NULL|Ama Qamata, Khosi...|
|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|
|Jailbirds New Orl...|           NULL|                NULL|
|        Kota Factory|           NULL|Mayur More, Jiten...|
+--------------------+---------------+--------------------+
only showing top 5 rows



#### Organizar Select

In [15]:
df.select('title', 'type', 'country').show(5)

+--------------------+-------+-------------+
|               title|   type|      country|
+--------------------+-------+-------------+
|Dick Johnson Is Dead|  Movie|United States|
|       Blood & Water|TV Show| South Africa|
|           Ganglands|TV Show|         NULL|
|Jailbirds New Orl...|TV Show|         NULL|
|        Kota Factory|TV Show|        India|
+--------------------+-------+-------------+
only showing top 5 rows



#### Filtrar DF
**Se tiver espaços, você deve utilizar a função col no nome da coluna**

In [24]:
# df.filter('country = "Brazil"').show()
df.filter(col('cast') == 'Afonso Padilha').show()

+-------+-----+--------------------+--------------------+--------------+-------+-----------------+------------+------+--------+---------------+--------------------+
|show_id| type|               title|            director|          cast|country|       date_added|release_year|rating|duration|      listed_in|         description|
+-------+-----+--------------------+--------------------+--------------+-------+-----------------+------------+------+--------+---------------+--------------------+
|  s2051|Movie|Afonso Padilha: C...|Junior Carelli, R...|Afonso Padilha| Brazil|September 3, 2020|        2020| TV-MA|  63 min|Stand-Up Comedy|Brazilian comedia...|
+-------+-----+--------------------+--------------------+--------------+-------+-----------------+------------+------+--------+---------------+--------------------+



#### Filtrar com duas condições (AND/&)

In [40]:
df.filter((col('country') == 'Brazil') & (col('type') == 'Movie') & (col('release_year') == '2021')).show()

+-------+-----+-------------+-------------+--------------------+-------+--------------+------------+------+--------+--------------------+--------------------+
|show_id| type|        title|     director|                cast|country|    date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+-------------+-------------+--------------------+-------+--------------+------------+------+--------+--------------------+--------------------+
|   s784|Movie|     Carnaval| Leandro Neri|Giovana Cordeiro,...| Brazil|  June 2, 2021|        2021| TV-MA|  95 min|Comedies, Interna...|After a breakup, ...|
|   s967|Movie|Get the Grift|Pedro Antonio|Marcus Majella, S...| Brazil|April 28, 2021|        2021| TV-MA|  95 min|Comedies, Interna...|After a botched s...|
|  s1189|Movie| Get the Goat| Vitor Brandt|Matheus Nachterga...| Brazil|March 18, 2021|        2021| TV-14|  98 min|Action & Adventur...|Two hapless cops ...|
+-------+-----+-------------+-------------+---

#### Filtrar com 2 condições (OR/|)

In [52]:
df.filter((col('director') == 'Theodore Melfi') | (col('director') == 'Andy Devonshire')).show()

+-------+-------+--------------------+---------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|       country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|     s9|TV Show|The Great British...|Andy Devonshire|Mel Giedroyc, Sue...|United Kingdom|September 24, 2021|        2021| TV-14|9 Seasons|British TV Shows,...|A talented batch ...|
|    s10|  Movie|        The Starling| Theodore Melfi|Melissa McCarthy,...| United States|September 24, 2021|        2021| PG-13|  104 min|    Comedies, Dramas|A woman adjusting...|
+-------+-------+--------------------+---------------+--------------------+--------------+

#### Filtrar combinando & e |

In [56]:
df.filter((col('director') == 'Theodore Melfi') & (col('type') == 'Movie') | (col('country') == 'South Africa') & (col('type') == 'Movie')).show(5)

+-------+-----+--------------------+--------------+--------------------+-------------+------------------+------------+------+--------+--------------------+--------------------+
|show_id| type|               title|      director|                cast|      country|        date_added|release_year|rating|duration|           listed_in|         description|
+-------+-----+--------------------+--------------+--------------------+-------------+------------------+------------+------+--------+--------------------+--------------------+
|    s10|Movie|        The Starling|Theodore Melfi|Melissa McCarthy,...|United States|September 24, 2021|        2021| PG-13| 104 min|    Comedies, Dramas|A woman adjusting...|
|   s294|Movie|                Slay|     Adze Ugah|Enhle Mbali, Rams...| South Africa|    August 8, 2021|        2021| TV-MA|  86 min|Comedies, Interna...|In pursuit of bot...|
|   s765|Movie|Trippin' with the...| Jayan Moodley|Jailoshini Naidoo...| South Africa|      June 4, 2021|        20

#### Criando novas colunas (usando a função lit)

In [ ]:
df.withColumn('streaming', lit('Netflix')).show(5)
# Também é possível passar funções dentro do lit
df.withColumn('higher_2020', lit(col('release_year') > 2019)).show(20)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+-----------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|higher_2020|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+-----------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|       true|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After cro

#### Criar coluna condicional (usando função substring)

In [ ]:
df.withColumn('id_without_s', substring('show_id', 2, 5)).show(30)

+--------------------+-------------+---------------------------------+--------------------+----------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+------------+
|             show_id|         type|                            title|            director|                  cast|             country|          date_added|     release_year|              rating|            duration|           listed_in|         description|id_without_s|
+--------------------+-------------+---------------------------------+--------------------+----------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+------------+
|                  s1|        Movie|             Dick Johnson Is Dead|     Kirsten Johnson|                  NULL|       United States|  September 25, 2021|             2020|          

#### Criar coluna condicional (concat/concat_ws)

In [ ]:
df.withColumn('release_duration', concat('duration', 'release_year')).select('release_duration').show(5) #Como podemos reparar, sem espaços

+----------------+
|release_duration|
+----------------+
|      90 min2020|
|   2 Seasons2021|
|    1 Season2021|
|    1 Season2021|
|   2 Seasons2021|
+----------------+
only showing top 5 rows



In [ ]:
df.withColumn('release_duration', concat_ws(' - ', 'duration', 'release_year')).select('release_duration').show(5)
#Como vemos, com o separador determinado

+----------------+
|release_duration|
+----------------+
|   90 min - 2020|
|2 Seasons - 2021|
| 1 Season - 2021|
| 1 Season - 2021|
|2 Seasons - 2021|
+----------------+
only showing top 5 rows



#### Alterar tipo de coluna

In [76]:
df = df.withColumn('release_year', col('release_year').cast(IntegerType()))

In [79]:
df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



#### Usando funções lambdas

In [ ]:
onlyDateAdded = udf(lambda date: date.split(',')[0] if date else None)

df.withColumn('day', onlyDateAdded('date_added')).show(5)
# Esse arquivo tem dados nulos e infelizmente nao to conseguindo utilizar a função lambda para separar as datas